***Setting up the keys***

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  
Optionally, if you'd like to also use DeepSeek, create an account [here](https://platform.deepseek.com/), create a key [here](https://platform.deepseek.com/api_keys) and top up with at least the minimum $2 [here](https://platform.deepseek.com/top_up).

Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

***Import libraries***

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

#import google.generativeai #run if needed

***Load environment***

In [4]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key not set
Anthropic API Key not set
Google API Key not set


***Establish Connection***

In [ ]:
openai = OpenAI()

claude = anthropic.Anthropic()

#google.generativeai.configure() #run if needed

**temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic. 0 means more literal and 1 means creativity on steroids

In [ ]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [ ]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
completion = openai.chat.completions.create(model='gpt-4o-mini', messages=prompts)
print(completion.choices[0].message.content)

In [ ]:
completion = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

In [ ]:
completion = openai.chat.completions.create(
    model='gpt-4.1',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

In [ ]:
completion = openai.chat.completions.create(
    model='o3-mini',
    messages=prompts
)
print(completion.choices[0].message.content)

In [ ]:
# Claude 3.7 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [ ]:
# Claude 3.7 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

***A rare problem with Claude streaming on Windows***

Claude's streaming into Jupyter Lab's output -- it sometimes seems to eat up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

***The API for Gemini***


gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash-preview-04-17",
    messages=prompts
)
print(response.choices[0].message.content)

***Deepseek***

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(content.split(" ")))

***an adversarial conversation between Chatbots***

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [ ]:
# a conversation between GPT-4o-mini and Claude-3-haiku

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

This code defines a Python function called call_gpt.

Here's a breakdown:

def call_gpt():: This line defines the function named call_gpt. It doesn't take any arguments, suggesting it might rely on variables defined outside the function's scope.
messages = [{"role": "system", "content": gpt_system}]: Initializes a list called messages. The first item in the list is a dictionary representing a system message. The content of this message is taken from a variable named gpt_system, which is expected to be defined elsewhere and likely contains instructions for the GPT model.
for gpt, claude in zip(gpt_messages, claude_messages):: This loop iterates through pairs of messages from two variables, gpt_messages and claude_messages. zip combines elements from both lists so that in each iteration, gpt will be an element from gpt_messages and claude will be an element from claude_messages. These variables are likely lists of strings or dictionaries representing previous turns in a conversation with two different models (presumably GPT and Claude).
messages.append({"role": "assistant", "content": gpt}): Inside the loop, this line appends a dictionary to the messages list. This dictionary represents a message from the "assistant" (likely a previous response from the GPT model), with its content taken from the gpt variable in the current iteration.
messages.append({"role": "user", "content": claude}): This line also appends a dictionary to the messages list. This dictionary represents a message from the "user" (likely a previous response from the Claude model or a user's input), with its content taken from the claude variable in the current iteration.
completion = openai.chat.completions.create(...): This line makes a call to the create method of the chat.completions object from the openai library. This is where the function interacts with the GPT language model API.
model=gpt_model: Specifies the GPT model to use for the completion. The model name is taken from the gpt_model variable, which is expected to be defined elsewhere.
messages=messages: Provides the list of messages for the conversation history that was constructed in the previous steps.
return completion.choices[0].message.content: This line returns the content of the message from the GPT model's response. completion.choices is a list of possible completions, and [0] selects the first (and usually the only) choice. .message.content extracts the actual text of the generated message from the model.
In summary, the call_gpt function constructs a conversation history by interleaving system instructions and previous messages from two sources (gpt_messages and claude_messages), sends this history to a specified GPT model using the openai library, and returns the generated response from the GPT model. This pattern is often used in scenarios where you are orchestrating a conversation between multiple language models or incorporating previous turns into a new request.

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

This code defines a Python function called call_claude.

Here's a breakdown:

def call_claude():: This line defines the function named call_claude. Like call_gpt, it doesn't take any arguments directly, suggesting it relies on external variables.
messages = []: Initializes an empty list called messages. This list will be populated with the conversation history for the Claude model.
for gpt, claude_message in zip(gpt_messages, claude_messages):: This loop iterates through pairs of messages from gpt_messages and claude_messages using zip. In each iteration, gpt will be an element from gpt_messages and claude_message will be an element from claude_messages.
messages.append({"role": "user", "content": gpt}): Inside the loop, this line appends a dictionary to the messages list. This dictionary represents a message from the "user", with its content taken from the gpt variable. This is interesting because it suggests that the responses from the GPT model (gpt_messages) are being treated as "user" input when constructing the conversation for the Claude model.
messages.append({"role": "assistant", "content": claude_message}): This line appends a dictionary to the messages list. This dictionary represents a message from the "assistant" (likely a previous response from the Claude model), with its content taken from the claude_message variable.
messages.append({"role": "user", "content": gpt_messages[-1]}): After the loop, this line appends one more message to the messages list. This message is from the "user" and its content is the last element of the gpt_messages list (gpt_messages[-1]). This seems to be adding the final response from the GPT model as the last user turn in the conversation history for Claude.
message = claude.messages.create(...): This line makes a call to the create method of the messages object from the claude library (presumably the Anthropic Claude API library). This is where the function interacts with the Claude language model API.
model=claude_model: Specifies the Claude model to use for the completion. The model name is taken from the claude_model variable, which is expected to be defined elsewhere.
system=claude_system: Provides a system prompt specifically for the Claude model. The content is taken from the claude_system variable, expected to be defined elsewhere and likely contains instructions for the Claude model.
messages=messages: Provides the list of messages for the conversation history that was constructed.
max_tokens=500: Sets the maximum number of tokens (words or parts of words) that the Claude model should generate in its response.
return message.content[0].text: This line returns the text content of the Claude model's response. The structure of the response from the Claude API is slightly different from the OpenAI API. message.content is a list of content blocks, and [0].text accesses the text content of the first block.
In summary, the call_claude function constructs a conversation history for the Claude model by interleaving previous messages from gpt_messages and claude_messages, with the GPT messages often treated as user input for Claude. It includes a specific system prompt for Claude, sends this history to a specified Claude model using the claude library, and returns the generated text response from the Claude model. This function seems designed for a scenario where you are managing a conversation that involves both GPT and Claude models, potentially having them respond to each other's outputs.

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
This code snippet sets up and runs a simple conversational exchange between what appear to be two different language models, referred to as "GPT" and "Claude".

Here's a breakdown:

gpt_messages = ["Hi there"]: Initializes a list called gpt_messages with an initial greeting from the "GPT" side of the conversation.
claude_messages = ["Hi"]: Initializes a list called claude_messages with an initial greeting from the "Claude" side.
print(f"GPT:\n{gpt_messages[0]}\n"): Prints the initial message from GPT.
print(f"Claude:\n{claude_messages[0]}\n"): Prints the initial message from Claude.
for i in range(5):: This loop runs the conversational exchange 5 times.
gpt_next = call_gpt(): Inside the loop, this line calls the call_gpt() function (defined in a previous cell) to get the next response from the GPT model. The generated response is stored in the gpt_next variable.
print(f"GPT:\n{gpt_next}\n"): Prints the new response from GPT.
gpt_messages.append(gpt_next): Appends the new GPT response to the gpt_messages list, adding it to the conversation history for GPT.
claude_next = call_claude(): Calls the call_claude() function (defined in a previous cell) to get the next response from the Claude model. The generated response is stored in the claude_next variable.
print(f"Claude:\n{claude_next}\n"): Prints the new response from Claude.
claude_messages.append(claude_next): Appends the new Claude response to the claude_messages list, adding it to the conversation history for Claude.
In essence, this code simulates a back-and-forth conversation for 5 turns, where each model's response is generated based on the ongoing conversation history with both models, and the turns are printed to the output.

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)